# what to generate... 
* add option to have some translations in native language...
# website with html, java
* toggle "include", insert file path --> insert file, host it as a website..
### add lisardo csv automatic instructions

... a system, where you can check every line, and regenerate if necessary...

In [6]:
# works with the csv export from the lln website

import pandas as pd
import re
from dotenv import load_dotenv
import os
import openai
import random

# chatGPT parameters

load_dotenv()
openai.api_key = os.getenv("22API_KEY")

max_tokens = 100
max_words = max_tokens * 0.3
print("maximum words:", max_words)
model = "gpt-4-1106-preview" # "gpt-3.5-turbo-16k # gpt-4-1106-preview # gpt-4-turbo # gpt-4-32k

# what should be generated by chatGPT?

native_language = "english"

global include_definition
global include_synonyms
global include_explanation
global include_hint
global native_language_include_definition
global native_language_include_synonyms
global native_language_include_explanation
global native_language_include_hint

include_definition = False
native_language_include_definition = True

include_synonyms = False
native_language_include_synonyms = True

include_hint = False
native_language_include_hint = True

include_explanation = False
native_language_include_explanation = False

# Check if the CSV file exists
csv_file_path = "/Users/moritzvitt/Downloads/df_gpt.csv"
if not os.path.exists(csv_file_path):
    # Create a new CSV file
    with open(csv_file_path, "w") as csv_file:
        # Write the header row
        csv_file.write("synonyms, hints, definition, explanation, native_language_definition, native_language_explanation")

df_gpt = pd.read_csv(csv_file_path, delimiter='\t')

print(df_gpt.columns)
#building the data frame
df = pd.read_csv('/Users/moritzvitt/downloads/anki/items.csv', delimiter='\t')


df.columns = df.iloc[0]
df = df.drop(df.iloc[:, [1, 7, 11, 12, 13, 14, 15, 16]], axis=1)

df.columns = ["ID", "short_phrase", "short_translation", "word", "stem", "word_type", "word_translation", "website", "lang", "source", "date", "long_phrase", "machine_translation", "human_translation", "first_jpg", "second_jpg", "audio"]

print(df.columns)
print(df.head)

maximum words: 30.0
Index(['synonyms, hints, definition, explanation, native_language_definition, native_language_explanation'], dtype='object')


ValueError: Length mismatch: Expected axis has 15 elements, new values have 17 elements

# First of all: SYNONYMS!

In [ ]:
if include_synonyms == True: 
    df['synonyms'] = ''
    def findSynonyms(lang, stem, usage):
        if lang == "en":
            language = "english"
        if lang == "it":
            language = "italian"
        if lang == "fr":
            language = "french"
        if lang == "ja":
            language = "japanese"
            
        #if not synonyms:
        messages = [        
            {"role": "user", "content": f"Provide me with two synonyms for '{stem}' in {language} that truly match\
            the meaning of '{stem}' in '{usage}'.\
            If there is no synonym, that fits the meaning of '{stem}' in the context of '{usage}', you can provide with a short description of '{stem}' instead.\
            Provide the synonyms always separated with a comma inbetween.\
            Don't repeat '{stem}' or 'synonym: 'at the beginning of your answer and don't put numbers or bullet points before the synonyms you return\
            Use a maximum of {max_words} words for each synonym."}
        ]

        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            #temperature=0.5
        )
        
        synonyms = response.choices[0].message["content"]
        print(synonyms) 
        
        return synonyms

    df["synonyms"] = df.apply(lambda row: findSynonyms(row["lang"], row["stem"], row["long_phrase"]), axis=1)

In [ ]:
if native_language_include_synonyms == True: 
    df['synonyms'] = ''
    def native_language_findSynonyms(stem, usage):
        #if not synonyms:
        messages = [        
            {"role": "user", "content": f"Provide me with two synonyms for '{stem}' in {native_language} that truly match\
            the meaning of '{stem}' in '{usage}'.\
            If there is no synonym, that fits the meaning of '{stem}' in the context of '{usage}', you can provide with a short description of '{stem}' instead.\
            Provide the synonyms always separated with a comma inbetween.\
            Don't repeat '{stem}' or 'synonym: 'at the beginning of your answer and don't put numbers or bullet points before the synonyms you return\
            Use a maximum of {max_words} words for each synonym."}
        ]

        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            #temperature=0.5
        )
        
        synonyms = response.choices[0].message["content"]
        print(stem, "   :", synonyms) 
        
        return synonyms

    df["synonyms"] = df.apply(lambda row: native_language_findSynonyms(row["stem"], row["long_phrase"]), axis=1)

asapissimo    : pronto, immediately
Unterlage    : documents, paperwork
Brutto    : gross, before taxes
verbeamten    : tenure, civil servant status
Gehalt    : salary, income
merkwürdige    : odd, peculiar
inkompetent    : inept, unqualified
Turnhalle    : gymnasium, sports hall
Kabelbrand    : electrical fire, wire fire
Strafanzeige    : criminal complaints, police reports
Herzinfarkt    : myocardial infarction, heart attack
fahl    : pallid, lackluster
kratzen    : scratch, itch
Faust    : fist, clenched hand
beeilen    : hurry, hasten
begeben    : proceed, move
Glühbirnen    : incandescent bulbs, light bulbs
Mistviecher    : brats, pests
bescheuert    : ridiculous, absurd
Schlankheitsfarm    : weight loss clinic, slimming spa
pennen    : crash, sleep over
Vorgeschmack    : foretaste, preview
schuldest    : owe, are indebted
anheben    : raise, hoist
verlegen    : install, lay
Glitter    : sparkle, shimmer
Arschritze    : butt crack, gluteal cleft
verarschen    : mock, fool
übernimm

# hint

In [ ]:
if include_hint == True: 
    df['hint'] = ''
    def create_hint(lang, synonyms, word, stem, usage):
        length = len(usage.split())
        # print(length)
        max_tokens = int(150 + length + 0.33 * length)
        max_words = 8
        if lang == "en":
        #if not synonyms:
            messages = [
                {"role": "user", "content": 
                f"Provide me with a brief explanation of '{word}'. You can also take in account the context of '{stem}' in '{usage}'.\
                Be very concise, use a maximum of {max_words} words!\
                Return the explanation in the form '{word}: explanation.'."}
            ]

        if lang == "fr": 
            messages = [
                {"role": "user", "content": 
                f"Fournissez-moi une brève explication de {word}'\
                Vous pouvez également tenir compte du contexte de '{stem}' dans '{usage}'.\
                Soyez très concis, utilisez un maximum de {max_words} mots ! Je veux que l'explication soit différente de '{synonyms}'.\
                Renvoyez l'explication sous la forme '{word} : explication."}
            ]

        if lang == "it": 
            messages = [
                {"role": "user", "content": 
                f"Fornisci una breve spiegazione di '{word}'.\
                Si può anche prendere in considerazione il contesto di '{stem}' in '{usage}'.\
                Siate molto concisi, usare un massimo di {max_words} parole!\
                Restituire la spiegazione nella forma '{word}: spiegazione.'"}
            ]

        if lang == "ja": 
            messages = [
                {"role": "user", "content": 
                f"'{word}'の簡単な説明を提供してください。また、'{usage}'における'{stem}'の文脈を考慮することもできます。\
                非常に簡潔に表現し、最大で{max_words}単語を使用してください。\
                説明は'{word}：説明。'の形式で提供してください."}
            ]
            
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            #temperature=0.5
        )
        
        hint = response.choices[0].message["content"]
        print(hint) 
        
        return hint

    df["hint"] = df.apply(lambda row: create_hint(row["lang"], row["synonyms"], row["word"], row["stem"], row["long_phrase"]), axis=1)

# native Language hint

In [ ]:
if native_language_include_hint == True: 
    df['hint'] = ''
    def create_hint(synonyms, word, stem, usage):
        length = len(usage.split())
        # print(length)
        max_tokens = int(150 + length + 0.33 * length)
        max_words = 8
        if native_language == "english":
        #if not synonyms:
            messages = [
                {"role": "user", "content": 
                f"Provide me with a brief explanation of '{word}'. You can also take in account the context of '{stem}' in '{usage}'.\
                Be very concise, use a maximum of {max_words} words!\
                Return the explanation in the form '{word}: explanation.'."}
            ]
        
        if native_language == "turkish":
        #if not synonyms:
            messages = [
                {"role": "user", "content": 
                f"'{word}' kelimesinin kısa bir açıklamasını sağla. \
                    Ayrıca, '{usage}' içinde '{stem}' bağlamını da dikkate alabilirsin. \
                    Çok özlü ol, en fazla {max_words} kelime kullan!\
                    Açıklamayı '{word}: açıklama.' şeklinde döndür."}
            ]

        if native_language == "french": 
            messages = [
                {"role": "user", "content": 
                f"Fournissez-moi une brève explication de {word}'\
                Vous pouvez également tenir compte du contexte de '{stem}' dans '{usage}'.\
                Soyez très concis, utilisez un maximum de {max_words} mots ! Je veux que l'explication soit différente de '{synonyms}'.\
                Renvoyez l'explication sous la forme '{word} : explication."}
            ]

        if native_language == "italian": 
            messages = [
                {"role": "user", "content": 
                f"Fornisci una breve spiegazione di '{word}'.\
                Si può anche prendere in considerazione il contesto di '{stem}' in '{usage}'.\
                Siate molto concisi, usare un massimo di {max_words} parole!\
                Restituire la spiegazione nella forma '{word}: spiegazione.'"}
            ]

        if native_language == "japanese": 
            messages = [
                {"role": "user", "content": 
                f"'{word}'の簡単な説明を提供してください。また、'{usage}'における'{stem}'の文脈を考慮することもできます。\
                非常に簡潔に表現し、最大で{max_words}単語を使用してください。\
                説明は'{word}：説明。'の形式で提供してください."}
            ]
            
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            #temperature=0.5
        )
        
        hint = response.choices[0].message["content"]
        print(hint) 
        
        return hint

    df["hint"] = df.apply(lambda row: create_hint(row["synonyms"], row["word"], row["stem"], row["long_phrase"]), axis=1)

asapissimo: extremely urgent, ASAP intensified.
Unterlagen: Documents required, to be submitted later.
Brutto: Gross amount before deductions like taxes.
verbeamtet: granted civil servant status.
Gehalt: Usually means "salary" in German.
Merkwürdige: "Strange" or "peculiar" in context.
inkompetent: lacking necessary skills or knowledge.
Turnhalle: Sports or gymnasium hall.
Kabelbrand: electrical cable fire or malfunction.
Strafanzeigen: formal criminal complaints or reports.
Herzinfarkt: German for heart attack.
fahlen: having a pallid, negative connotation.
kratzen: German for 'scratch'.
Faust: A symbol for force or violent threat.
beeilt: imperative of 'hurry up'.
begeben: to proceed, go to a place.
Glühbirnen: German for lightbulbs, electric bulbs for illumination.
Mistviecher: colloquial, derogatory for animals/pests.
bescheuert: foolish or senseless.
Schlankheitsfarm: weight loss retreat or slimming spa.
pennen: German slang for sleeping or crashing.
Vorgeschmack: A small preview 

# include definition

In [ ]:
if include_definition == True: 
    df['definition'] = ''
    def create_definition(lang, word, stem, usage):
        length = len(usage.split())
        # print(length)
        max_tokens = int(150 + length + 0.33 * length)
        max_words = 15
        if lang == "en":
        #if not synonyms:
            messages = [
                {"role": "user", "content": 
                f"Provide me with a dictionary like definition of '{word}'. You can also take in account the context of '{stem}' in '{usage}'.\
                Be very concise, use a maximum of {max_words} words!\
                Return the definition in the form '{word}: your definition.'."}
            ]

        if lang == "fr": 
            messages = [
                {"role": "user", "content": 
                f"Fournissez-moi une brève définition de '{word}' semblable à celle d'un dictionnaire.\
                Vous pouvez également tenir compte du contexte de '{stem}' dans '{usage}'.\
                Soyez très concis, utilisez un maximum de {max_words} mots !\
                Renvoyez la définition sous la forme '{word} : votre définition."}
            ]

        if lang == "it": 
            messages = [
                {"role": "user", "content": 
                f"Fornite una breve definizione da dizionario di '{word}' .\
                Si può anche prendere in considerazione il contesto di '{stem}' in '{usage}'.\
                Siate molto concisi, usare un massimo di {max_words} parole!\
                Restituire la definizione nella forma '{word}: la vostra definizione.'"}
            ]
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            #temperature=0.5
        )
        
        hint = response.choices[0].message["content"]
        print(hint) 
        
        return hint

    df["definition"] = df.apply(lambda row: create_definition(row["lang"], row["word"], row["stem"], row["long_phrase"]), axis=1)

# native_language definition

In [ ]:
if native_language_include_definition == True: 
    df['definition'] = ''
    def create_definition(word, stem, usage, word_type):
        length = len(usage.split())
        # print(length)
        max_tokens = int(150 + length + 0.33 * length)
        max_words = 15
        if native_language == "english":
        #if not synonyms:
            messages = [
                {"role": "user", "content": 
                f"Provide me with a dictionary like definition of '{word}'. You can also take in account the context of '{stem}' in '{usage}'.\
                Be very concise, use a maximum of {max_words} words!\
                Return the definition in the form '{word}: your definition.'."}
            ]

        if native_language == "turkish":
            # instruction = ""
            # # if word_type == "Noun":
            # #     instruction = ""
            # # if word_type == "Adj":
            # if word_type == "Verb":
            #     instruction = "(sonsuz kip)"
        #if not synonyms:
            messages = [
                {"role": "user", "content": 
                f"'{word}' için sözlük tarzında bir tanım sağla. \
                    Ayrıca, '{usage}' içinde '{stem}' bağlamını da dikkate alabilirsin. \
                    Çok özlü ol, en fazla {max_words} kelime kullan!\
                    Tanımı '{word}: tanımınız.' şeklinde döndür."}
            ]

        if native_language == "french": 
            messages = [
                {"role": "user", "content": 
                f"Fournissez-moi une brève définition de '{word}' semblable à celle d'un dictionnaire.\
                Vous pouvez également tenir compte du contexte de '{stem}' dans '{usage}'.\
                Soyez très concis, utilisez un maximum de {max_words} mots !\
                Renvoyez la définition sous la forme '{word} : votre définition."}
            ]

        if native_language == "italian": 
            messages = [
                {"role": "user", "content": 
                f"Fornite una breve definizione da dizionario di '{word}' .\
                Si può anche prendere in considerazione il contesto di '{stem}' in '{usage}'.\
                Siate molto concisi, usare un massimo di {max_words} parole!\
                Restituire la definizione nella forma '{word}: la vostra definizione.'"}
            ]
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            #temperature=0.5
        )
        
        hint = response.choices[0].message["content"]
        print(hint) 
        
        return hint

    df["definition"] = df.apply(lambda row: create_definition(row["word"], row["stem"], row["long_phrase"], row["word_type"]), axis=1)

asapissimo: Extremely informal, exaggerated version of "as soon as possible," implying utmost urgency.
Unterlagen: documents or paperwork required for a specific process or transaction.
Brutto: Total income before deductions such as taxes and social security contributions.
verbeamtet: holding the status of a civil servant, typically with employment benefits and job security.
Gehalt: 1. Content, substance. 2. Salary, wages. (Context suggests the playful use of ambiguity.)
Merkwürdige: adjective, meaning strange, odd, or peculiar in the given context.
inkompetent: lacking the necessary skills or knowledge to perform effectively; unqualified.
Turnhalle: Gymnasium or sports hall, often part of a school or community center.
Kabelbrand: Electrical cable fire caused by short-circuiting or overheating.
Strafanzeigen: Formal reports of criminal activities filed to the authorities.
Herzinfarkt: Sudden blockage in coronary artery, causing heart muscle damage or death; myocardial infarction.
fahle

In [ ]:
def createID(ID, source, date):
    newID = f"{ID}{source}{date}"
    newID = newID.replace(" ", "__")
    return newID
df["ID"] = df.apply(lambda row: createID(row["ID"], row["source"], row["date"]), axis=1)

def mp3_pattern(mp3):
        mp3 = "[sound:" + mp3 + "]"
        return mp3
df["audio"] = df.apply(lambda row: mp3_pattern(row["audio"]), axis=1)

def image(first_jpg, second_jpg):
    image = f'<img src="{first_jpg}"><img src="{second_jpg}">'
    return image
df["first_jpg"] = df.apply(lambda row: image(row["first_jpg"], row["second_jpg"]), axis=1)

# def clozePattern(word, phrase, translations):
#         word_new = f"{{c1::{word}::{translations}}}"
#         word_new = str(word_new)
#         #print(word_new)
#         phrase = phrase.replace(word, word_new)
#         return phrase
#         print(phrase)
# df["long_phrase"] = df.apply(lambda row: clozePattern(row["word"], row["long_phrase"], row["synonyms"]), axis=1)

import re

def clozePattern(word, phrase, translations):
    word_new = f"{{{{c1::{word}::{translations}}}}}"
    phrase = re.sub(rf'\b{word}\b', word_new, phrase, flags=re.IGNORECASE)
    return phrase

df["long_phrase"] = df.apply(lambda row: clozePattern(row["word"], row["long_phrase"], row["synonyms"]), axis=1)

def cleanHint(hint):
    # print(type(hint))
    if hint != "" and ":" in hint:
        parts = re.findall(r"[:](.+)?", hint) 
        # r"[:](.+)?" worked
        return f": {parts[0]}"
    else: 
        return ""
df["hint"] = df["hint"].apply(cleanHint)

# def definition_field(definition):
#       newDefinition = definition
#       return newDefinition
# df["definition"] = df.apply(lambda row: definition_field(row["word_translation"]), axis=1)

def notes_field(translation, word_translation):
    translation_str = str(translation) if not pd.isnull(translation) else ""
    word_translation_str = str(word_translation) if not pd.isnull(word_translation) else ""
    newNotes = translation_str + word_translation_str
    return newNotes

df["notes"] = df.apply(lambda row: notes_field(row["human_translation"], row["word_translation"]), axis=1)


In [ ]:
ankiOut=df[['ID', 'long_phrase', 'hint', 'definition', 'first_jpg', 'audio', 'synonyms', 'notes']].copy()
ankiOut.columns = ['ID', 'cloze', 'hint', 'definition', 'image', 'audio', 'synonyms', 'notes']
# ankiOut.to_csv("/Users/moritzvitt/coding/anki.csv", sep=';', index=False, header=False)
ankiOut.to_csv("/Users/moritzvitt/coding/anki.csv", sep='\t', index=False)


df.to_excel("/Users/moritzvitt/coding/df.xlsx", index=False, header=False)
